In [144]:
import pandas as pd
import numpy as np
import requests
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import folium
import math
import heapq

In [146]:
here_api = 'TRwPRsFs9Xi1qtNQ0gbOuc_bg8GO7oVdWlPbzPtw_JE'

In [148]:
#get the latitude and longitutde using HERE maps of the list of cities provided
def get_lat_long_here(city_list,here_api):
    # Provide your HERE maps api key 
    API_KEY = here_api
    # Initiate an empty list to store the lat, long of the cities and their names
    lat_long = []
    # Loop over all cities and store the lat, long from the response using the HERE api
    for city in city_list:
        url = f'https://geocode.search.hereapi.com/v1/geocode?q={city},India&apiKey={API_KEY}'
        response = requests.get(url).json()
        #print(response)
        if 'items' in response:
            location = response['items'][0]['position']
            name = response['items'][0]['title']
            city_data = {
                'city_name':name,
                'latitude':location['lat'],
                'longitude':location['lng']
            } 
            lat_long.append(city_data)
        else:
            continue
    # Return the final list
    return lat_long


In [150]:
city_list = ['Mumbai','Delhi','Jabalpur','Bangalore','Kolkata','Chennai','Ahmedabad','Indore','Hyderabad']

In [152]:
city_lat_long = get_lat_long_here(city_list,here_api)

In [153]:
city_lat_long

[{'city_name': 'Mumbai, Maharashtra, India',
  'latitude': 18.94018,
  'longitude': 72.83484},
 {'city_name': 'Delhi, India', 'latitude': 28.63412, 'longitude': 77.21688},
 {'city_name': 'Jabalpur, Madhya Pradesh, India',
  'latitude': 23.17416,
  'longitude': 79.93132},
 {'city_name': 'Bengaluru, Karnataka, India',
  'latitude': 12.96614,
  'longitude': 77.58694},
 {'city_name': 'Kolkata, West Bengal, India',
  'latitude': 22.5706,
  'longitude': 88.37132},
 {'city_name': 'Chennai, Tamil Nadu, India',
  'latitude': 13.08377,
  'longitude': 80.28256},
 {'city_name': 'Ahmedabad, Gujarat, India',
  'latitude': 23.02778,
  'longitude': 72.60032},
 {'city_name': 'Indore, Madhya Pradesh, India',
  'latitude': 22.71622,
  'longitude': 75.86509},
 {'city_name': 'Hyderabad, Telangana, India',
  'latitude': 17.3949,
  'longitude': 78.47081}]

In [154]:
##calculate the distance matrix of the cities in the list provided
def get_distance_here_maps(api_key, origin, destination):
    # Prepare the API URL for Distance Matrix
    base_url = "https://router.hereapi.com/v8/routes"
    
    # Extract latitude and longitude from origin and destination
    origin_coords = f"{origin['latitude']},{origin['longitude']}"
    destination_coords = f"{destination['latitude']},{destination['longitude']}"

    # API parameters
    params = {
        'transportMode': 'car',
        'origin': origin_coords,
        'destination': destination_coords,
        'return': 'summary',
        'apikey': api_key
    }

    # Make the request to HERE Maps API
    response = requests.get(base_url, params=params)
    data = response.json()

    # Extract distance in meters
    if response.status_code == 200:
        distance_meters = data['routes'][0]['sections'][0]['summary']['length']
        distance_km = distance_meters / 1000  # Convert meters to kilometers
        return distance_km
    else:
        print(f"Error: {data}")
        return None
        
# Create distance matrix
n = len(city_lat_long)
distance_matrix = [[0] * n for _ in range(n)]
api_key = here_api
    
# Calculate distance between all combinations of cities
for i in range(n):
    for j in range(n):
        if i != j:
            distance_matrix[i][j] = get_distance_here_maps(api_key,
                city_lat_long[i],city_lat_long[j]
            )

In [155]:
distance_matrix = np.array(distance_matrix)
print(distance_matrix)

[[   0.    1453.688 1062.022  991.542 1902.186 1325.618  535.747  599.852
   722.954]
 [1431.208    0.     843.897 2128.998 1471.084 2164.788  979.472  820.631
  1537.87 ]
 [1060.308  850.077    0.    1365.528 1121.895 1401.318  895.455  505.874
   773.898]
 [ 988.501 2141.07  1371.469    0.    1874.828  347.517 1488.617 1356.387
   573.681]
 [1901.605 1547.279 1119.202 1876.       0.    1666.928 2075.824 1623.349
  1491.554]
 [1342.996 2172.466 1402.865  348.477 1667.001    0.    1828.405 1438.673
   629.273]
 [ 545.919  970.244  896.884 1490.962 2076.851 1824.038    0.     385.044
  1184.412]
 [ 599.711  820.691  507.241 1350.936 1624.289 1443.21   385.33     0.
   812.856]
 [ 712.386 1546.967  777.366  573.241 1491.609  629.378 1188.272  798.432
     0.   ]]


In [158]:
## Nearest Neighbor Insertion algorithm implementation
def nearest_insertion_tsp_complete(distance_matrix):
    num_cities = len(distance_matrix)
    visited = [False] * num_cities
    tour = []
    total_distance = 0

    # Start with the first city (index 0)
    current_city = 0
    visited[current_city] = True
    tour.append(current_city)

    # Find the nearest unvisited city from the starting city
    nearest_city = None
    min_distance = float('inf')
    for city in range(num_cities):
        if not visited[city] and distance_matrix[current_city][city] < min_distance:
            min_distance = distance_matrix[current_city][city]
            nearest_city = city

    # Add the nearest city to the tour
    visited[nearest_city] = True
    tour.append(nearest_city)
    total_distance += min_distance

    # Build the remaining tour
    while sum(visited) < num_cities:
        best_insertion_cost = float('inf')
        best_insertion_index = -1
        best_city = -1

        # Find the unvisited city and its best insertion point
        for city in range(num_cities):
            if visited[city]:
                continue
            for i in range(len(tour) - 1):
                insertion_cost = (
                    distance_matrix[tour[i]][city]
                    + distance_matrix[city][tour[i + 1]]
                    - distance_matrix[tour[i]][tour[i + 1]]
                )
                if insertion_cost < best_insertion_cost:
                    best_insertion_cost = insertion_cost
                    best_insertion_index = i
                    best_city = city

        # Insert the best city into the tour
        visited[best_city] = True
        total_distance += best_insertion_cost
        tour.insert(best_insertion_index + 1, best_city)

    # Complete the cycle by returning to the starting city
    total_distance += distance_matrix[tour[-1]][tour[0]]
    
    tour.append(tour[0])
    return tour, total_distance

In [97]:
# Example Usage
#distance_matrix = np.array(distance_matrix)
tour = nearest_insertion_tsp_complete(distance_matrix)
print("Tour:", [city_list[i] for i in tour[0]])
print("Total Distance:", tour[1])

Tour: ['Mumbai', 'Bangalore', 'Chennai', 'Hyderabad', 'Jabalpur', 'Kolkata', 'Delhi', 'Indore', 'Ahmedabad', 'Mumbai']
Total Distance: 7166.8859999999995


In [159]:
#modifying the Prim's algorithm to solve the TSP problem
def prims_tsp_open(distance_matrix):
    num_cities = len(distance_matrix)
    visited = [False] * num_cities
    tour = []
    total_distance = 0
    
    # Start from Node 0
    current_node = 0
    visited[current_node] = True
    tour.append(current_node)
    
    while sum(visited) < num_cities:
        min_distance = float('inf')
        next_node = None
        
        # Find the nearest unvisited neighbor
        for neighbor in range(num_cities):
            if not visited[neighbor] and distance_matrix[current_node][neighbor] < min_distance:
                min_distance = distance_matrix[current_node][neighbor]
                next_node = neighbor
        
        # Add the nearest neighbor to the tour
        visited[next_node] = True
        tour.append(next_node)
        total_distance += min_distance
        current_node = next_node
    
    return tour, total_distance

In [61]:
# Example Usage
tour_prim = prims_tsp(distance_matrix)
print("Tour:", [city_list[i] for i in tour_prim[0]])
print("Total Distance:", tour_prim[1])

([0, 6, 7, 2, 8, 3, 5, 4, 1, 0], 7769.924000000001)
Tour: ['Mumbai', 'Ahmedabad', 'Indore', 'Jabalpur', 'Hyderabad', 'Bangalore', 'Chennai', 'Kolkata', 'Delhi', 'Mumbai']
Total Distance: 7769.924000000001


In [222]:
# Traveling Salesman Problem using Branch and Bound.
maxsize = float('inf')

# Function to copy temporary solution to the final solution
# Function to copy temporary solution to the final solution
def copyToFinal(curr_path):
    final_path[:N + 1] = curr_path[:]
    final_path[N] = curr_path[0]

# Function to find the minimum edge cost 
# having an end at the vertex i
def firstMin(adj, i):
	min = maxsize
	for k in range(N):
		if adj[i][k] < min and i != k:
			min = adj[i][k]

	return min

# function to find the second minimum edge 
# cost having an end at the vertex i
def secondMin(adj, i):
	first, second = maxsize, maxsize
	for j in range(N):
		if i == j:
			continue
		if adj[i][j] <= first:
			second = first
			first = adj[i][j]

		elif(adj[i][j] <= second and
			adj[i][j] != first):
			second = adj[i][j]

	return second

# function that takes as arguments:
# curr_bound -> lower bound of the root node
# curr_weight-> stores the weight of the path so far
# level-> current level while moving in the search space tree
# curr_path[] -> where the solution is being stored which would later be copied to final_path[]
def TSPRec(adj, curr_bound, curr_weight, 
			level, curr_path, visited):
	global final_res
	
	# base case is when we have reached level N 
	# which means we have covered all the nodes once
	if level == N:
		
		# check if there is an edge from
		# last vertex in path back to the first vertex
		if adj[curr_path[level - 1]][curr_path[0]] != 0:
			
			# curr_res has the total weight
			# of the solution we got
			curr_res = curr_weight + adj[curr_path[level - 1]]\
										[curr_path[0]]
			if curr_res < final_res:
				copyToFinal(curr_path)
				final_res = curr_res
		return

	# for any other level iterate for all vertices
	# to build the search space tree recursively
	for i in range(N):
		
		# Consider next vertex if it is not same 
		# (diagonal entry in adjacency matrix and 
		# not visited already)
		if (adj[curr_path[level-1]][i] != 0 and
							visited[i] == False):
			temp = curr_bound
			curr_weight += adj[curr_path[level - 1]][i]

			# different computation of curr_bound 
			# for level 2 from the other levels
			if level == 1:
				curr_bound -= ((firstMin(adj, curr_path[level - 1]) +
								firstMin(adj, i)) / 2)
			else:
				curr_bound -= ((secondMin(adj, curr_path[level - 1]) +
								firstMin(adj, i)) / 2)

			# curr_bound + curr_weight is the actual lower bound 
			# for the node that we have arrived on.
			# If current lower bound < final_res, 
			# we need to explore the node further
			if curr_bound + curr_weight < final_res:
				curr_path[level] = i
				visited[i] = True
				
				# call TSPRec for the next level
				TSPRec(adj, curr_bound, curr_weight, 
					level + 1, curr_path, visited)

			# Else we have to prune the node by resetting 
			# all changes to curr_weight and curr_bound
			curr_weight -= adj[curr_path[level - 1]][i]
			curr_bound = temp

			# Also reset the visited array
			visited = [False] * len(visited)
			for j in range(level):
				if curr_path[j] != -1:
					visited[curr_path[j]] = True

# This function sets up final_path
def TSP(adj):
    # Calculate initial lower bound for the root node 
    # using the formula 1/2 * (sum of first min + 
    # second min) for all edges. Also initialize the 
    # curr_path and visited array
    global final_path, final_res, N

    curr_bound = 0
    curr_path = [-1] * (N + 1)
    visited = [False] * N

    # Compute initial bound
    for i in range(N):
        curr_bound += (firstMin(adj, i) +
                       secondMin(adj, i))

    # Rounding off the lower bound to an integer
    curr_bound = math.ceil(curr_bound / 2)

    # We start at vertex 1 so the first vertex 
    # in curr_path[] is 0
    visited[0] = True
    curr_path[0] = 0

    # Call to TSPRec for curr_weight 
    # equal to 0 and level 1
    TSPRec(adj, curr_bound, 0, 1, curr_path, visited)
    
    # Return the final path and cost
    return final_path, final_res


In [224]:
# Driver code

# Adjacency matrix for the given graph
N = len(city_list)
# final_path[] stores the final solution 
# i.e. the // path of the salesman.
final_path = [None] * (N + 1)

# visited[] keeps track of the already
# visited nodes in a particular path
visited = [False] * N

# Stores the final minimum weight
# of shortest tour.
final_res = maxsize

#TSP(distance_matrix)
#print("Tour:", [city_list[i] for i in final_path])
#print("Minimum cost :", final_res)

In [242]:
def print_solution(distance_matrix, city_list):
    #Branch & bound Algorithm
    # Adjacency matrix for the given graph
    N = len(city_list)
    # final_path[] stores the final solution 
    # i.e. the // path of the salesman
    final_path = [None] * (N + 1)

    # visited[] keeps track of the already
    # visited nodes in a particular path
    visited = [False] * N

    # Stores the final minimum weight of shortest tour
    final_res = maxsize
    

    final_path = TSP(distance_matrix)
    
    print("Tour for Branch & Bound Algorithm:", [city_list[i] for i in final_path[0]])
    print("Minimum cost for Branch & Bound Algorithm :", final_path[1])

    tour_prim = prims_tsp(distance_matrix)
    print("Tour for Prims algorithm:", [city_list[i] for i in tour_prim[0]])
    print("Total Distance for Prims Algorithm:", tour_prim[1])

    
    tour_nearest_insertion = nearest_insertion_tsp_complete(distance_matrix)
    print("Tour for Nearest insertion algorithm :", [city_list[i] for i in tour_nearest_insertion[0]])
    print("Total Distance for Nearest insertion algorithm:", tour_nearest_insertion[1])    


    

In [244]:
print_solution(distance_matrix,city_list)

Tour for Branch & Bound Algorithm: ['Mumbai', 'Ahmedabad', 'Indore', 'Delhi', 'Jabalpur', 'Kolkata', 'Chennai', 'Bangalore', 'Hyderabad', 'Mumbai']
Minimum cost for Branch & Bound Algorithm : 7008.745999999999
Tour for Prims algorithm: ['Mumbai', 'Ahmedabad', 'Indore', 'Jabalpur', 'Hyderabad', 'Bangalore', 'Chennai', 'Kolkata', 'Delhi', 'Mumbai']
Total Distance for Prims Algorithm: 7768.1759999999995
Tour for Nearest insertion algorithm : ['Mumbai', 'Bangalore', 'Chennai', 'Hyderabad', 'Jabalpur', 'Kolkata', 'Delhi', 'Indore', 'Ahmedabad', 'Mumbai']
Total Distance for Nearest insertion algorithm: 7166.7519999999995
